In [1]:
import time, math, pdb
import numpy as np
import pandas as pd
from numba import jit, njit, vectorize, float64, cuda
import seaborn as sns

In [2]:
@vectorize([float64(float64)])
def activation(x):
    return 1 / (1 + math.exp(-1*x))

@vectorize([float64(float64)])
def dactivation(x):
    return activation(x)*(1-activation(x))

@njit
def simulate(weights, concepts, noactivation = False):
    res = np.empty((0))
    for i, concept in enumerate(concepts):
        vweights = weights[:,i]
        vweights = vweights[~np.isnan(vweights)]
        vconcepts = np.delete(concepts, i)
        vweights = np.ascontiguousarray(vweights)
        vconcepts = np.ascontiguousarray(vconcepts)
        pot = np.dot(vweights, vconcepts)
        if noactivation:
            res = np.append(res, pot)
        else:
            res = np.append(res, activation(pot))
    return res

@njit
def activate(arr):
    for a in range(len(arr)):
        arr[a] = activation(arr[a])
    return arr

@njit
def deltaRule(data, weights, lc):
    for i in range(1, data.shape[0]):
        lr = lc
        pots = simulate(weights, data[i-1], True)
        error = data[i] - activate(pots)
        for a in range(len(weights)):
            weightarr = weights[a]
            for b in range(len(weightarr)):
                weight = weightarr[b]
                if not np.isnan(weight):
                    weights[a][b] += lr * (error[b] * dactivation(pots[b])) * data[i-1][a]
    return weights

@njit
def generateData(SAMPLES,  ref_weights):
    trainingData = np.empty((0,SIZE))
#     trainingData = []
    for i in range(SAMPLES):
#         trainingData.append(np.random.uniform(0,1, SIZE))
#         trainingData.append(simulate(ref_weights, trainingData[-1]))
        trainingData = np.append(trainingData, np.random.uniform(0,1, SIZE).reshape(1,SIZE), axis=0)
        trainingData = np.append(trainingData, simulate(ref_weights, trainingData[-1]).reshape(1,SIZE), axis=0)
    return trainingData

In [7]:
SAMPLES = 20000
# SIZE = len(stocks)
SIZE = 6
LEARNING_COEFFICIENT = 0.2
MIN_WEIGHT = -2
MAX_WEIGHT = 2
np.random.seed(2)

ref_concepts = np.zeros(SIZE)
ref_weights = (np.random.randint(MIN_WEIGHT*10, (MAX_WEIGHT*10)+1, SIZE**2)/10).reshape(SIZE,SIZE)
np.fill_diagonal(ref_weights, np.nan)

res_concepts = np.zeros(SIZE)
res_weights = (np.zeros(SIZE**2)).reshape(SIZE,SIZE)
np.fill_diagonal(res_weights, np.nan)

start = time.time()
trainingData = generateData(SAMPLES, ref_weights)
print(trainingData)
# trainingData = np.array(list(zip(*list(res.values()))))
end = time.time()
print(f"Generation completed in {round((end-start)*1000,2)} ms")

start = time.time()
deltaRule(trainingData, res_weights, LEARNING_COEFFICIENT)
end = time.time()
print(f"Training completed in {round((end-start)*1000,2)} ms")

print()
print(ref_weights)
print()
print(res_weights)
# print("\t", end= "")
# for item in stocks:
#     print(item + "↑\t", end="")
# print()
# processed = []
# for a, row in enumerate(res_weights):
#     tmp = []
#     for b, item in enumerate(row):
#         tmp.append(res_weights[a][b]/res_weights[b][a])
#     processed.append(tmp)
# cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)
# sns.heatmap(processed, annot=True, cmap=cmap)
# for i, row in enumerate(res_weights):
#     print(stocks[i] + "→\t", end = "")
#     for item in row:
#         print(str(round(item,2)) + "\t", end="")
#     print()
# print()
# print("\t", end= "")
# for item in stocks:
#     print(item + "↑\t", end="")
# print()
    
# for i, row in enumerate(processed):
#     print(stocks[i] + "→\t", end = "")
#     for item in row:
#         print(str(round(item,2)) + "\t", end="")
#     print()

[[0.28526305 0.44706645 0.68808789 0.62838708 0.31311637 0.11422091]
 [0.91968647 0.30670925 0.71724532 0.76653272 0.81489021 0.07717606]
 [0.74901045 0.7311007  0.14442611 0.78463502 0.45523177 0.84492156]
 ...
 [0.91655549 0.20802314 0.77780933 0.68781398 0.8794489  0.06243012]
 [0.61413726 0.89234066 0.57502176 0.06568526 0.10171798 0.3849875 ]
 [0.9430179  0.3275158  0.57941453 0.83638104 0.40999601 0.17323618]]
Generation completed in 7448.61 ms
Training completed in 407.95 ms

[[ nan -0.5 -1.2  0.2 -0.2 -0.9]
 [ 2.   nan  1.4  1.1 -0.9  0.1]
 [ 1.1  0.6  nan  1.7  1.9 -1.7]
 [ 1.8 -1.6  1.9  nan  1.3 -1.7]
 [-1.5  0.4 -1.6 -1.4  nan -0.1]
 [ 1.1 -1.8 -0.4 -0.8 -1.6  nan]]

[[        nan  0.52097877 -0.60876893 -0.21796912 -1.02456315  1.1929919 ]
 [ 1.98691407         nan  0.97542075  1.24003913 -0.3063502  -2.11080933]
 [ 0.40405132  0.37800193         nan  1.25381089  1.21679784 -1.28169763]
 [ 0.12699317 -1.04471238  1.67755604         nan  0.74439194 -0.60548126]
 [-1.1847421

In [ ]:
from pandas_datareader import data as pdr
import yfinance as yf
from datetime import datetime, timedelta


stocks = ["GOOG", "GOOGL", "BRK-A", "BRK-B", "AAL", "DAL", "GLU", "RYAAY"]
data = yf.download(stocks, start="2011-03-01", end="2020-08-21", interval="1d", group_by="ticker", threads=True)

In [ ]:
res = {}
for stock in stocks:
    start = data[stock]['High'].iloc[0]
    res[stock] = []
    for end in data[stock]['High']:
        res[stock].append((end-start)/start)
    res[stock] = res[stock]*1
